In [46]:
import cv2
import os
import numpy as np
from yoloface import face_analysis
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [47]:
face = face_analysis()

yolov3-tiny_face.weights:: status : file already exists
yolov3_tiny_face.cfg:: status : file already exists
face_detection.weights:: status : file already exists
face_detection.cfg:: status : file already exists


In [2]:
cap = cv2.VideoCapture('C:/Users/admin/Desktop/UNIPA/MAGISTRALE/appunti_universita/PRIMO_ANNO/VISIONE_ARTIFICIALE/ASSIGMENT/assigment_4/video_antonio_2.mp4')

i = 1
while True:
    ret, frame = cap.read()
    if not ret:
        print("Video non trovato o la lettura è terminata")
        break

    _, boxes, confidences = face.face_detection(frame_arr=frame, frame_status=True, model='full')

    for box in boxes:
        # Estrae le coordinate del box rilevato
        x, y, w, h = box
        print("Coordinate del box " + str(i) + ": x={}, y={}, w={}, h={}".format(x, y, w, h))

        # Verifica che il rettangolo del volto sia all'interno dei limiti dell'immagine
        if x >= 0 and y >= 0 and w >= 0 and h >= 0:
            # Ritaglia l'immagine per ottenere solo il volto
            img_r = frame[y:y+w, x:x+h]

            # Verifica se l'immagine ritagliata è valida
            if img_r.size != 0:
                # Percorso di output per l'immagine ritagliata
                output_path = "C:/Users/admin/Desktop/UNIPA/MAGISTRALE/appunti_universita/PRIMO_ANNO/VISIONE_ARTIFICIALE/ASSIGMENT/assigment_4/Antonio_S/Antonio_S{}.png".format(i)

                # Salva l'immagine del volto ritagliato
                #cv2.imwrite(output_path, img_r)

                # Incrementa il contatore
                i += 1
            else:
               print("Immagine ritagliata non valida")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


yolov3-tiny_face.weights:: status : file already exists
yolov3_tiny_face.cfg:: status : file already exists
face_detection.weights:: status : file already exists
face_detection.cfg:: status : file already exists
Coordinate del box 1: x=212, y=123, w=284, h=196
Coordinate del box 2: x=208, y=124, w=282, h=202
Coordinate del box 3: x=209, y=124, w=283, h=199
Coordinate del box 4: x=207, y=123, w=285, h=201
Coordinate del box 5: x=188, y=126, w=283, h=228


KeyboardInterrupt: 

Preparo il dataset con le immagini e le etichette corrispondenti

In [48]:
x_dataset = []  # Lista per memorizzare le immagini
y_dataset = []  # Lista per memorizzare le etichette corrispondenti


for dirpath, dirnames, filenames in os.walk('dataset/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirpath, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (28, 28))
        x_dataset.append(img)

        y_dataset.append(dirpath[8:])

x_dataset = np.asarray(x_dataset).astype('float32') / 255. - 0.5
y_dataset = np.asarray(y_dataset)


Creo la rete siamese

In [49]:
def euclidean_distance(d):
    return K.sum(K.square(d[0] - d[1]), axis=-1)

In [50]:
def get_branch(shape_input):
    input_img = keras.Input(shape=shape_input)
    # Uno strato convoluzionale con 64 filtri,di dimensione 3x3'
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    #Uno strato di MaxPooling con una finestra di dimensione 2x2'
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    y = layers.Flatten()(x)
    model = keras.Model(input_img, y)
    return model

In [51]:
def get_siamese(shape_input=(28, 28, 1)):
    branch = get_branch(shape_input)
    input_img_1 = keras.Input(shape=shape_input)# rappresenta il primo input dell'immagine per la rete Siamese. La forma dell'input viene definita da shape_input
    input_img_2 = keras.Input(shape=shape_input)
    des_1 = branch(input_img_1)
    des_2 = branch(input_img_2)#des_1 e des_2, rappresentano le rappresentazioni delle caratteristiche delle rispettive immagini.
    output = layers.Lambda(euclidean_distance, name='output_layer')([des_1, des_2])#viene calcolata la distanza euclidea tra le caratteristiche
    model = keras.Model([input_img_1, input_img_2] , output)
    model.summary()
    return model

Viene definita una funzione di loss.
Viene specificata una metrica di valutazione aggiuntiva per monitorare le prestazioni del modello durante l'addestramento e la validazione. In questo caso, viene utilizzata la metrica di errore quadratico medio (tf.keras.metrics.MeanSquaredError()), che misura la discrepanza media tra i valori di verità e le previsioni del modello.

In [52]:
#compiling#
model = get_siamese()

def contrastive_loss(y_true, y_pred, margin = 1):
    return y_true * y_pred + (1.0 - y_true) * tf.math.maximum(margin - y_pred, 0.0)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.MeanSquaredError()])

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 model_4 (Functional)           (None, 784)          7264        ['input_8[0][0]',                
                                                                  'input_9[0][0]']                
                                                                                                  
 output_layer (Lambda)          (None,)              0           ['model_4[0][0]',          

 Il data generator è una funzione di generazione di dati per addestrare un modello. Prende in input un insieme di dati di addestramento (x_train e y_train) e una dimensione del batch (batch_size).
 Genera casualmente degli indici (indici) all'interno dell'intervallo degli indici di x_train. Successivamente, estrae un sottoinsieme di x_train e y_train corrispondente agli indici generati (sub_set e sub_set_y).
  crea coppie di dati (pairs) combinando ogni elemento di sub_set con tutti gli elementi successivi a esso.
  L'etichetta è impostata su 0 se gli elementi delle coppie sono uguali, altrimenti è impostata su 1.
  Infine, vengono creati due array (x1 e x2) che contengono rispettivamente il primo e il secondo elemento di ogni coppia di dati.

In [53]:
def data_gen(x_train, y_train, batch_size):
    while True:
        indici = np.random.randint(0, x_train.shape[0], batch_size)
        sub_set = x_train[indici]
        sub_set_y = y_train[indici]
        pairs = [[a, b] for idx, a in enumerate(sub_set) for b in sub_set[idx + 1:]]
        label = np.asarray([ 0 if a==b else 1 for idx, a in enumerate(sub_set_y) for b in sub_set_y[idx + 1:]])
        x1 = np.asarray([ a[0] for a in pairs])
        x2 = np.asarray([ a[1] for a in pairs])
        yield (x1, x2), label.astype('float32')

In questa parte si dividono i dati in set di addestramento e validazione, creando generatori di dati per entrambi i set, si definisce una callback di stop anticipato e quindi addestra il modello utilizzando i generatori di dati.
 Si controlla la funzione 'val_loss' e interrompendo l'addestramento se il valore non migliora per un numero (patience) specificato di epoche.

In [54]:
x_train, x_val, y_train, y_val = train_test_split(x_dataset, y_dataset, test_size=0.2)

data_gen_batch=data_gen(x_train, y_train, 32)
data_gen_batch_validate=data_gen(x_val, y_val, 32)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(x = data_gen_batch, epochs=10, steps_per_epoch=100, shuffle=True, validation_data=data_gen_batch_validate,  validation_steps=10, callbacks=[callback])

Epoch 1/10
100/100 [==============================] - 39s 380ms/step - loss: 0.7277 - mean_squared_error: 0.2256 - val_loss: 0.9218 - val_mean_squared_error: 0.1851
Epoch 2/10
100/100 [==============================] - 38s 376ms/step - loss: 0.7681 - mean_squared_error: 0.2360 - val_loss: 0.6471 - val_mean_squared_error: 0.1615
Epoch 3/10
100/100 [==============================] - 38s 379ms/step - loss: 0.5505 - mean_squared_error: 0.1766 - val_loss: 0.4370 - val_mean_squared_error: 0.1497
Epoch 4/10
100/100 [==============================] - 39s 389ms/step - loss: 0.4890 - mean_squared_error: 0.1768 - val_loss: 0.7387 - val_mean_squared_error: 0.2629
Epoch 5/10
100/100 [==============================] - 38s 379ms/step - loss: 0.7461 - mean_squared_error: 0.2469 - val_loss: 0.5405 - val_mean_squared_error: 0.1829
Epoch 6/10
100/100 [==============================] - 38s 379ms/step - loss: 0.4494 - mean_squared_error: 0.1386 - val_loss: 0.4205 - val_mean_squared_error: 0.1366
Epoch 7/10

La funzione predict prende in input un modello addestrato (model), un dato di test (x_test), un insieme di dati (x_dataset) e le relative etichette (y_dataset).
 viene creato un elenco di coppie di dati (pairs) combinando il dato di test con ciascun elemento dell'insieme di dati x_dataset
 Viene quindi creato un array x1 contenente il primo elemento di ogni coppia di dati e un array x2 contenente il secondo elemento di ogni coppia di dati.
Successivamente, viene effettuata una previsione del modello utilizzando gli array x1 e x2 come input
Dopo aver ottenuto tutte le previsioni, viene identificato l'indice del valore minimo nella lista pred (contenente le distanze). Questo indice viene quindi utilizzato per recuperare l'etichetta corrispondente dall'insieme di dati y_dataset,
Infine, l'etichetta corrispondente all'indice con la previsione minima viene restituita come risultato della funzione.

In [55]:
def predict(model, x_test, x_dataset, y_dataset):

    pairs=[[x_test, b] for b in x_dataset]

    x1 = np.asarray([ a[0] for a in pairs])
    x2 = np.asarray([ a[1] for a in pairs])


    pred = (model.predict((x1, x2)))
    #print("pred: ", pred)


    return y_dataset[np.argmin(pred)]

In [56]:
indici = np.random.randint(0, x_dataset.shape[0], 50)
subset_x = x_dataset[indici]
subset_y = y_dataset[indici]

In [57]:
def preprocessig(img):
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img, (28,28))
    img=img.astype('float32') / 255. - 0.5
    return img

In [59]:
video = cv2.VideoCapture('test.mp4')

# Ottenere le informazioni del video
video_FourCC = int(video.get(cv2.CAP_PROP_FOURCC))
fps = video.get(cv2.CAP_PROP_FPS)
H = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_size = (W, H)

# Creare il video di output
output = cv2.VideoWriter('Video_Finale5.mp4', video_FourCC, fps, frame_size)

if video is None or not video.isOpened() or output is None or not output.isOpened():
    print("--(!) Error opening video capture")
    exit(0)

ret, frame = video.read()
while ret:
    if frame is None:
        break

    _, boxes, confidences = face.face_detection(frame_arr=frame, frame_status=True, model='full')

    for k in range(len(boxes)):

        x, y, w, h = boxes[k][0], boxes[k][1], boxes[k][2], boxes[k][3]

        c_x = x + h // 2
        c_y = y + w // 2
        l = max(w, h)
        new_x = c_x - 1 // 2
        new_y = c_y - 1 // 2
        img_r = frame[new_y:new_y + 1, new_x:new_x + 1]

        if img_r.size != 0:
            img_r = preprocessig(img_r)
            pred = predict(model, img_r, subset_x, subset_y)
            #print("pred2: ",pred)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 4)
            cv2.putText(frame, pred, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    output.write(frame)
    ret, frame = video.read()

video.release()
output.release()



2/2 [==============================] - 0s 15ms/step
